<a href="https://colab.research.google.com/github/koyarekoikoi/tkpj/blob/main/skin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tqdm import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Define paths
metadata_path = "/content/drive/MyDrive/SKIN/HAM10000_metadata.csv"
images_path_1 = "/content/drive/MyDrive/SKIN/HAM10000_images_part_1"
images_path_2 = "/content/drive/MyDrive/SKIN/HAM10000_images_part_2"

In [5]:
# Load metadata
df = pd.read_csv(metadata_path)

In [6]:
# Image processing function
def load_images(image_ids, image_folder, target_size=(128, 128)):
    images = []
    for img_id in tqdm(image_ids):
        img_path = os.path.join(image_folder, img_id + ".jpg")
        if os.path.exists(img_path):
            img = cv2.imread(img_path)
            img = cv2.resize(img, target_size)
            img = img / 255.0  # Normalize
            images.append(img)
    return np.array(images)

In [7]:
# Load images from both folders
X_images = load_images(df['image_id'], images_path_1)
X_images2 = load_images(df['image_id'], images_path_2)
X = np.concatenate([X_images, X_images2])

100%|██████████| 10015/10015 [36:53<00:00,  4.53it/s]


In [8]:
# Remaining Imports to load model, can be done on init import
import pickle
from tensorflow.keras.models import load_model


In [9]:
# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['dx'])

In [10]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [11]:
# Define CNN model
def build_cnn_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(len(np.unique(y)), activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


In [12]:
# Train CNN
cnn_model = build_cnn_model()
cnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
cnn_accuracy = cnn_model.evaluate(X_test, y_test)[1]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


NotFoundError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run

  File "/usr/local/lib/python3.11/dist-packages/tornado/ioloop.py", line 699, in <lambda>

  File "/usr/local/lib/python3.11/dist-packages/tornado/ioloop.

In [ ]:
# Save CNN model
cnn_model.save("cnn_skin_cancer.h5")

In [ ]:
# Extract features for ML models
X_flat = X.reshape(X.shape[0], -1)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_flat)
pca = PCA(n_components=100)
X_pca = pca.fit_transform(X_scaled)

In [ ]:
# Train ML models
rf_model = RandomForestClassifier(n_estimators=100)
rf_model.fit(X_pca[:len(y_train)], y_train)
y_pred_rf = rf_model.predict(X_pca[len(y_train):])
rf_accuracy = accuracy_score(y_test, y_pred_rf)

svm_model = SVC(kernel='linear')
svm_model.fit(X_pca[:len(y_train)], y_train)
y_pred_svm = svm_model.predict(X_pca[len(y_train):])
svm_accuracy = accuracy_score(y_test, y_pred_svm)


In [ ]:
# Save ML models
with open("rf_model.pkl", "wb") as f:
    pickle.dump(rf_model, f)
with open("svm_model.pkl", "wb") as f:
    pickle.dump(svm_model, f)
with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)
with open("pca.pkl", "wb") as f:
    pickle.dump(pca, f)

In [ ]:
# results
print(f"CNN Accuracy: {cnn_accuracy:.4f}")
print(f"Random Forest Accuracy: {rf_accuracy:.4f}")
print(f"SVM Accuracy: {svm_accuracy:.4f}")

In [ ]:
# Take image path input at runtime
image_path = input("Enter the path of the image: ")

# Choose model type (CNN, RF, or SVM)
model_type = input("Enter model type (cnn/rf/svm): ").lower()

# Make prediction
predict_custom_image(image_path, model_type)


In [ ]:
# Function to load a custom image and make a prediction
def predict_custom_image(image_path, model_type="cnn"):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (128, 128))
    img = img / 255.0  # Normalize
    img = np.expand_dims(img, axis=0)

    if model_type == "cnn":
        model = load_model("cnn_skin_cancer.h5")
        prediction = np.argmax(model.predict(img))
    else:
        with open("rf_model.pkl", "rb") as f:
            rf_model = pickle.load(f)
        with open("svm_model.pkl", "rb") as f:
            svm_model = pickle.load(f)
        with open("scaler.pkl", "rb") as f:
            scaler = pickle.load(f)
        with open("pca.pkl", "rb") as f:
            pca = pickle.load(f)

        img_flat = img.reshape(1, -1)
        img_scaled = scaler.transform(img_flat)
        img_pca = pca.transform(img_scaled)
        prediction = rf_model.predict(img_pca)[0] if model_type == "rf" else svm_model.predict(img_pca)[0]

    label = label_encoder.inverse_transform([prediction])[0]
    print(f"Predicted class: {label}")
    return label

In [ ]:
import gradio as gr

def predict_uploaded_image(image, model_type="cnn"):
    if image is None:
        return "No image uploaded!"

    img = cv2.imread(image)
    img = cv2.resize(img, (128, 128))
    img = img / 255.0  # Normalize
    img = np.expand_dims(img, axis=0)

    if model_type == "cnn":
        model = load_model("cnn_skin_cancer.h5")
        prediction = np.argmax(model.predict(img))
    else:
        with open("rf_model.pkl", "rb") as f:
            rf_model = pickle.load(f)
        with open("svm_model.pkl", "rb") as f:
            svm_model = pickle.load(f)
        with open("scaler.pkl", "rb") as f:
            scaler = pickle.load(f)
        with open("pca.pkl", "rb") as f:
            pca = pickle.load(f)

        img_flat = img.reshape(1, -1)
        img_scaled = scaler.transform(img_flat)
        img_pca = pca.transform(img_scaled)
        prediction = rf_model.predict(img_pca)[0] if model_type == "rf" else svm_model.predict(img_pca)[0]

    label = label_encoder.inverse_transform([prediction])[0]
    return f"Predicted class: {label}"

# Create Gradio UI
iface = gr.Interface(
    fn=predict_uploaded_image,
    inputs=[
        gr.Image(type="filepath"),
        gr.Radio(["cnn", "rf", "svm"], label="Choose Model Type", value="cnn")
    ],
    outputs="text"
)

iface.launch()
